In [42]:
from pyspark import RDD, SparkContext
from collections import defaultdict
from itertools import combinations
import math
import json
import time
from copy import deepcopy

In [58]:
case_number = 1
support = 15
input = "small2.csv"
output = "output.txt"

test_partitions = 2

In [44]:
sc = SparkContext("local", "HW2").getOrCreate()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=HW2, master=local) created by __init__ at <ipython-input-3-9ec08a7068e3>:1 

In [59]:
user_biz_RDD = sc.textFile(input, test_partitions).map(lambda line: line.split(','))

In [60]:
user_biz_RDD.getNumPartitions()

2

In [61]:
header = user_biz_RDD.first()

In [62]:
user_biz_RDD = user_biz_RDD.filter(lambda data: data!=header)

In [63]:
if case_number == 1:
    user_biz_RDD = user_biz_RDD.map(lambda x:(x[0],x[1])).groupByKey().map(lambda x: list(set(x[1])))
elif case_number == 2:
    user_biz_RDD = user_biz_RDD.map(lambda x:(x[1],x[0])).groupByKey().map(lambda x: list(set(x[1])))

In [64]:
total_baskets = user_biz_RDD.count()
total_baskets

49

## Phase 1

In [65]:
subset = user_biz_RDD.collect()

In [12]:
def pass1(subset, total_baskets, support):
    candidates = []
    kminusone_cands = []

    baskets = list(subset)
    chunk_size = float(float(len(subset))/float(total_baskets))
    ps = math.floor(float(chunk_size * support))
    print(ps)

    # Single Candidates

    singletons = defaultdict(int)
    for b in baskets:
        for item in b:
            singletons[item]+=1

    #singletons = sorted(singletons)
    #print(singletons)
    for k,v in singletons.items():
        if int(v)>=int(ps):
            candidates.append({k})
            kminusone_cands.append([k])

    kminusone_cands = sorted(kminusone_cands, key= lambda x: (len(x),x))
    group_size = 2
    intermediate_groups = combinations(kminusone_cands,group_size)
    new_groups = []
    for pair in intermediate_groups:
        new_groups.append(set().union(*pair))
    print(new_groups)

    group_size = 3
    while len(new_groups)>0:
    #for i in range(2):

        kplusonetest = []
        for k_cand in new_groups:
            candidates.append(k_cand)
        for pair in new_groups:
            #pair = set().union(*pair)
            pair_count = 0
            for b in baskets:
                if pair.issubset(b):
                    pair_count+=1
            if pair_count>ps:
                print(pair, pair_count)
                candidates.append(pair)
                kplusonetest.append(pair)
        if len(kplusonetest)==0: break;

        """for k_cand in kplusonetest:
            candidates.append(k_cand)
            print(k_cand)"""
        #group_size+=1
        new_groups = []


        for i in range(len(kplusonetest)):

            for j in range(i+1, len(kplusonetest)):
                #print(list(kplusonetest[i])[:group_size-1])
                #print(list(kplusonetest[j])[:group_size-1])
                if sorted(list(kplusonetest[i])[:group_size-2]) == sorted(list(kplusonetest[j])[:group_size-2]):

                    new_groups.append(kplusonetest[i].union(kplusonetest[j]))
        group_size+=1
        print(new_groups)



        """
        new_candidates = []

        if len(group_count)>0:
            next_set = defaultdict(int)
            for k_cand in group_count:
                candidates.append(k_cand)
                new_candidates.append(k_cand)
                for x in k_cand:
                    next_set[x]+=1

        print(new_candidates)

        kminusone_cands = []
        for cand,cand_count in next_set.items():
            kminusone_cands.append({cand})
        #print("CANDDIATES of size ", group_size)
        #print("Ewqeqweqwe" ,kminusone_cands)

        group_size+=1
        """
    return(candidates)

pass1(subset, total_baskets, support)

15
[{'39', '33'}, {'40', '33'}, {'46', '33'}, {'39', '40'}, {'39', '46'}, {'40', '46'}]


[{'46'},
 {'33'},
 {'40'},
 {'39'},
 {'33', '39'},
 {'33', '40'},
 {'33', '46'},
 {'39', '40'},
 {'39', '46'},
 {'40', '46'}]

In [13]:
def pass1(subset, total_baskets, cands, support):

    candidates = []

    baskets = list(subset)
    chunk_size = math.floor(len(baskets)/int(total_baskets))
    ps = chunk_size * support
    #print(ps)

    # Single Candidates

    singletons = defaultdict(int)
    for b in baskets:
        for item in b:
            singletons[item]+=1

    #print(singletons)
    for k,v in singletons.items():
        if v>=ps:
            candidates.append(frozenset({k}))
    new_singletons = candidates
    kminusone_cands = candidates
    group_size = 2
    while True:
        group_count = []

        for pair in combinations(new_singletons,group_size):
            print("groupsize: ", group_size)
            pair = frozenset().union(*pair)
            pair_count = 0
            for b in baskets:
                if pair.issubset(b):
                    pair_count+=1
            #print(pair_count)
            if pair_count>=ps:
                group_count.append(pair)
                #print(pair, pair_count)
        if len(group_count)==0: break;
        #print(group_count)
        if len(group_count)>0:
            next_set = defaultdict(int)
            for k_cand in group_count:
                candidates.append(k_cand)
                for x in k_cand:
                    next_set[x]+=1

        #print(next_set)

        kminusone_cands = []
        for cand,cand_count in next_set.items():
            kminusone_cands.append({cand})
        #print("CANDDIATES of size ", group_size)
        #print("Ewqeqweqwe" ,kminusone_cands)

        group_size+=1




    return(candidates)

pass1(subset, total_baskets, support)

TypeError: pass1() missing 1 required positional argument: 'support'

In [81]:
#WORKINGGGGG
def pass1(subset, cands, total_baskets, support):

    test_candidates = deepcopy(cands)
    subset = list(subset)
    chunk_size = float(float(len(subset))/float(total_baskets))
    ps = math.floor(float(chunk_size * support))
    print(ps)
    #print(cands)
    pass1candidates = []
    k = 2

    while len(cands) > 0:
    #for i in range(2):
        k_freq = []
        for candidate in test_candidates:
            cand_support = 0
            for basket in subset:
                if all(element in basket for element in candidate):
                    cand_support += 1

            if cand_support >= ps:
                k_freq.append(candidate)
                pass1candidates.append((tuple(candidate),1))

        if len(k_freq)==0:  break
        #pass1candidates.append(tuple(k_freq))


        kplusonecands = []
        for i in range(len(k_freq)):
            for j in k_freq[i+1:]:
                if k_freq[i][:k-2] == j[:k-2]:
                    x = sorted(list(set().union(k_freq[i],j)))
                    kplusonecands.append(x)
        print(kplusonecands)

        test_candidates = kplusonecands
        k+=1
        #print(kplusonecands)

    print(pass1candidates)
    return pass1candidates


x = pass1(subset, singletons, total_baskets, support)
x

3
[['100', '101'], ['100', '102'], ['100', '103'], ['100', '104'], ['100', '105'], ['100', '106'], ['100', '107'], ['100', '108'], ['100', '109'], ['100', '110'], ['100', '111'], ['100', '113'], ['100', '114'], ['100', '97'], ['100', '98'], ['100', '99'], ['101', '102'], ['101', '103'], ['101', '104'], ['101', '105'], ['101', '106'], ['101', '107'], ['101', '108'], ['101', '109'], ['101', '110'], ['101', '111'], ['101', '113'], ['101', '114'], ['101', '97'], ['101', '98'], ['101', '99'], ['102', '103'], ['102', '104'], ['102', '105'], ['102', '106'], ['102', '107'], ['102', '108'], ['102', '109'], ['102', '110'], ['102', '111'], ['102', '113'], ['102', '114'], ['102', '97'], ['102', '98'], ['102', '99'], ['103', '104'], ['103', '105'], ['103', '106'], ['103', '107'], ['103', '108'], ['103', '109'], ['103', '110'], ['103', '111'], ['103', '113'], ['103', '114'], ['103', '97'], ['103', '98'], ['103', '99'], ['104', '105'], ['104', '106'], ['104', '107'], ['104', '108'], ['104', '109'], [

[(('100',), 1),
 (('101',), 1),
 (('102',), 1),
 (('103',), 1),
 (('104',), 1),
 (('105',), 1),
 (('106',), 1),
 (('107',), 1),
 (('108',), 1),
 (('109',), 1),
 (('110',), 1),
 (('111',), 1),
 (('113',), 1),
 (('114',), 1),
 (('97',), 1),
 (('98',), 1),
 (('99',), 1),
 (('100', '103'), 1),
 (('100', '97'), 1),
 (('101', '102'), 1),
 (('101', '103'), 1),
 (('101', '104'), 1),
 (('101', '105'), 1),
 (('101', '106'), 1),
 (('101', '108'), 1),
 (('101', '97'), 1),
 (('101', '98'), 1),
 (('102', '103'), 1),
 (('102', '104'), 1),
 (('102', '105'), 1),
 (('102', '106'), 1),
 (('102', '107'), 1),
 (('102', '108'), 1),
 (('102', '109'), 1),
 (('102', '110'), 1),
 (('102', '111'), 1),
 (('102', '113'), 1),
 (('102', '114'), 1),
 (('102', '97'), 1),
 (('102', '98'), 1),
 (('102', '99'), 1),
 (('103', '104'), 1),
 (('103', '105'), 1),
 (('103', '106'), 1),
 (('103', '107'), 1),
 (('103', '108'), 1),
 (('103', '109'), 1),
 (('103', '110'), 1),
 (('103', '111'), 1),
 (('103', '113'), 1),
 (('103', '

In [90]:
#OG
def pass1(subset, total_baskets, support):
    candidates = []

    baskets = list(subset)
    chunk_size = float(float(len(baskets))/float(total_baskets))
    ps = math.floor(float(chunk_size * support))
    print(ps)

    # Single Candidates

    singletons = defaultdict(int)
    for b in baskets:
        for item in b:
            singletons[item]+=1

    kminusone_cands = []
    #print(singletons)
    for k,v in singletons.items():
        if v>=ps:
            candidates.append((tuple({k}),1))
            kminusone_cands.append({k})

    #kminusone_cands = []
    group_size = 2
    while True:
        group_count = []
        for pair in combinations(kminusone_cands,group_size):
            pair = frozenset().union(*pair)
            pair_count = 0
            for b in baskets:
                if pair.issubset(b):
                    pair_count+=1
            if pair_count>=ps:
                group_count.append(pair)
                #print(pair, pair_count)
        if len(group_count)==0: break;
        #print(group_count)
        if len(group_count)>0:
            next_set = defaultdict(int)
            for k_cand in group_count:
                candidates.append((tuple(k_cand),1))
                for x in k_cand:
                    next_set[x]+=1

        #print(next_set)

        kminusone_cands = []
        for cand,cand_count in next_set.items():
            kminusone_cands.append({cand})
        #print("CANDDIATES of size ", group_size)
        #print("Ewqeqweqwe" ,kminusone_cands)

        group_size+=1

    return(candidates)

pass1(subset[:10], total_baskets, support)

3


[(('107',), 1),
 (('97',), 1),
 (('103',), 1),
 (('100',), 1),
 (('105',), 1),
 (('106',), 1),
 (('108',), 1),
 (('104',), 1),
 (('101',), 1),
 (('98',), 1),
 (('113',), 1),
 (('102',), 1),
 (('110',), 1),
 (('111',), 1),
 (('99',), 1),
 (('109',), 1),
 (('114',), 1),
 (('107', '103'), 1),
 (('105', '107'), 1),
 (('106', '107'), 1),
 (('108', '107'), 1),
 (('107', '104'), 1),
 (('107', '102'), 1),
 (('107', '110'), 1),
 (('107', '109'), 1),
 (('97', '103'), 1),
 (('97', '100'), 1),
 (('105', '97'), 1),
 (('106', '97'), 1),
 (('97', '104'), 1),
 (('97', '101'), 1),
 (('97', '98'), 1),
 (('97', '102'), 1),
 (('100', '103'), 1),
 (('105', '103'), 1),
 (('106', '103'), 1),
 (('108', '103'), 1),
 (('104', '103'), 1),
 (('101', '103'), 1),
 (('103', '98'), 1),
 (('113', '103'), 1),
 (('102', '103'), 1),
 (('110', '103'), 1),
 (('111', '103'), 1),
 (('99', '103'), 1),
 (('109', '103'), 1),
 (('114', '103'), 1),
 (('105', '106'), 1),
 (('108', '105'), 1),
 (('105', '104'), 1),
 (('105', '101')

In [91]:
singletons = user_biz_RDD.flatMap(lambda x: x).sortBy(lambda x: x).distinct().map(lambda x: [x]).sortBy(lambda x:x).collect()
singletons

[['100'],
 ['101'],
 ['102'],
 ['103'],
 ['104'],
 ['105'],
 ['106'],
 ['107'],
 ['108'],
 ['109'],
 ['110'],
 ['111'],
 ['112'],
 ['113'],
 ['114'],
 ['115'],
 ['116'],
 ['117'],
 ['118'],
 ['119'],
 ['120'],
 ['121'],
 ['97'],
 ['98'],
 ['99']]

In [92]:
candidates = user_biz_RDD.mapPartitions(lambda partition: pass1(partition, total_baskets, support)).reduceByKey(lambda x,y:min(x,y)).map(lambda x: frozenset((x[0]))).sortBy(lambda x: (len(x),x)).collect()
candidates

[frozenset({'97'}),
 frozenset({'103'}),
 frozenset({'105'}),
 frozenset({'104'}),
 frozenset({'101'}),
 frozenset({'99'}),
 frozenset({'109'}),
 frozenset({'114'}),
 frozenset({'107'}),
 frozenset({'100'}),
 frozenset({'106'}),
 frozenset({'108'}),
 frozenset({'98'}),
 frozenset({'113'}),
 frozenset({'102'}),
 frozenset({'110'}),
 frozenset({'111'}),
 frozenset({'112'}),
 frozenset({'106', '107'}),
 frozenset({'107', '108'}),
 frozenset({'102', '107'}),
 frozenset({'103', '97'}),
 frozenset({'105', '97'}),
 frozenset({'104', '97'}),
 frozenset({'101', '97'}),
 frozenset({'97', '99'}),
 frozenset({'103', '105'}),
 frozenset({'103', '104'}),
 frozenset({'101', '103'}),
 frozenset({'103', '99'}),
 frozenset({'100', '98'}),
 frozenset({'100', '102'}),
 frozenset({'104', '105'}),
 frozenset({'101', '105'}),
 frozenset({'105', '99'}),
 frozenset({'106', '108'}),
 frozenset({'106', '98'}),
 frozenset({'106', '113'}),
 frozenset({'102', '106'}),
 frozenset({'106', '110'}),
 frozenset({'108', 

## Phase 2

In [93]:
def pass2(dataset, candidates):
    itemset_count = defaultdict(int)
    for basket in dataset:
        for candidate in candidates:
            if candidate.issubset(basket):
                itemset_count[candidate]+=1
    #print(candidate, itemset_count)

    final_item_set = []
    for key,value in itemset_count.items():
        final_item_set.append((key,value))

    return final_item_set

pass2(subset, candidates)

[(frozenset({'97'}), 34),
 (frozenset({'103'}), 28),
 (frozenset({'105'}), 23),
 (frozenset({'107'}), 21),
 (frozenset({'100'}), 30),
 (frozenset({'106'}), 22),
 (frozenset({'106', '107'}), 15),
 (frozenset({'103', '97'}), 18),
 (frozenset({'105', '97'}), 17),
 (frozenset({'103', '105'}), 15),
 (frozenset({'100', '107'}), 13),
 (frozenset({'100', '106'}), 13),
 (frozenset({'103', '107'}), 17),
 (frozenset({'105', '107'}), 11),
 (frozenset({'100', '97'}), 22),
 (frozenset({'106', '97'}), 14),
 (frozenset({'100', '103'}), 17),
 (frozenset({'103', '106'}), 16),
 (frozenset({'100', '105'}), 10),
 (frozenset({'105', '106'}), 12),
 (frozenset({'103', '106', '107'}), 13),
 (frozenset({'103', '105', '106'}), 9),
 (frozenset({'100', '105', '97'}), 10),
 (frozenset({'104'}), 24),
 (frozenset({'101'}), 26),
 (frozenset({'108'}), 22),
 (frozenset({'107', '108'}), 12),
 (frozenset({'104', '97'}), 14),
 (frozenset({'101', '97'}), 21),
 (frozenset({'103', '104'}), 14),
 (frozenset({'101', '103'}), 16

In [94]:
freq_item_set = user_biz_RDD.mapPartitions(lambda partition: pass2(partition, candidates)).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1]>=support).map(lambda x: x[0]).sortBy(lambda x:len(x)).collect()
    #groupByKey().map(lambda x:x[0]).collect()
freq_item_set

[frozenset({'103'}),
 frozenset({'106'}),
 frozenset({'101'}),
 frozenset({'108'}),
 frozenset({'110'}),
 frozenset({'114'}),
 frozenset({'97'}),
 frozenset({'105'}),
 frozenset({'107'}),
 frozenset({'100'}),
 frozenset({'104'}),
 frozenset({'98'}),
 frozenset({'99'}),
 frozenset({'113'}),
 frozenset({'102'}),
 frozenset({'109'}),
 frozenset({'106', '107'}),
 frozenset({'103', '97'}),
 frozenset({'103', '105'}),
 frozenset({'103', '107'}),
 frozenset({'100', '103'}),
 frozenset({'101', '97'}),
 frozenset({'100', '101'}),
 frozenset({'103', '99'}),
 frozenset({'101', '105'}),
 frozenset({'102', '106'}),
 frozenset({'102', '108'}),
 frozenset({'101', '99'}),
 frozenset({'102', '103'}),
 frozenset({'105', '108'}),
 frozenset({'106', '99'}),
 frozenset({'108', '99'}),
 frozenset({'101', '102'}),
 frozenset({'105', '97'}),
 frozenset({'100', '97'}),
 frozenset({'103', '106'}),
 frozenset({'101', '103'}),
 frozenset({'103', '108'}),
 frozenset({'100', '104'}),
 frozenset({'97', '98'}),
 froz

In [19]:
def write_output(candidates, freq_item_set):
    cand_list = []
    for x in candidates:
        cand_list.append(list(x))

    cand_list = sorted(cand_list, key= lambda x: (len(x),x))
    print(cand_list)

    sorted_item_set = []
    for x in freq_item_set:
        sorted_item_set.append(list(x))


    sorted_item_set = sorted(sorted_item_set, key= lambda x: (len(x),x))


    with open(output, 'w') as f:
        f.write("Candidatesx:")

        cur_len = 0
        for cand in cand_list:
            print(cand)
            l = len(cand)
            if l>cur_len:
                if l == 1:
                    towrite = "(\'" + str(cand[0])+"\')"
                    f.write("\n")
                else:
                    towrite = str(cand)
                    f.write("\n\n")
                f.write(towrite)
                cur_len=l
            else:
                if l == 1:
                    towrite = ",(\'" + str(cand[0])+"\')"
                    f.write(towrite)
                else:
                    cur_len=l
                    towrite = "," + str(cand)
                    f.write(towrite)





        f.write("\n\nFrequent Itemsets:")

        cur_len = 0
        for frq_is in sorted_item_set:
            frq_is = tuple(sorted(frq_is))
            l = len(frq_is)
            if l>cur_len:
                if l == 1:
                    towrite = "(\'" + str(frq_is[0])+"\')"
                    f.write("\n")
                else:
                    towrite = str(frq_is)
                    f.write("\n\n")
                f.write(towrite)
                cur_len=l
            else:
                if l == 1:
                    towrite = ",(\'" + str(frq_is[0])+"\')"
                    f.write(towrite)
                else:
                    cur_len=l
                    towrite = "," + str(frq_is)
                    f.write(towrite)


write_output(candidates, freq_item_set)

[['10'], ['11'], ['15'], ['16'], ['17'], ['23'], ['24'], ['25'], ['26'], ['28'], ['3'], ['33'], ['34'], ['35'], ['38'], ['39'], ['40'], ['41'], ['42'], ['43'], ['44'], ['46'], ['9'], ['11', '15'], ['11', '35'], ['11', '41'], ['11', '43'], ['11', '9'], ['17', '11'], ['17', '9'], ['34', '33'], ['34', '44'], ['35', '15'], ['39', '33'], ['41', '43'], ['42', '46'], ['11', '35', '15'], ['11', '41', '43'], ['17', '9', '11']]
['10']
['11']
['15']
['16']
['17']
['23']
['24']
['25']
['26']
['28']
['3']
['33']
['34']
['35']
['38']
['39']
['40']
['41']
['42']
['43']
['44']
['46']
['9']
['11', '15']
['11', '35']
['11', '41']
['11', '43']
['11', '9']
['17', '11']
['17', '9']
['34', '33']
['34', '44']
['35', '15']
['39', '33']
['41', '43']
['42', '46']
['11', '35', '15']
['11', '41', '43']
['17', '9', '11']


In [41]:
def write_output(candidates, freq_item_set):
    with open(output, 'w') as f:
        f.write("Candidates:\n")
        cand_list = defaultdict(list)
        for x in candidates:
            cand_list[len(x)].append((sorted(tuple(x))))

        for k, val in sorted(cand_list.items()):
            #print(k, sorted(val))
            init = True
            strn = ""
            if k==1:
                for k in sorted(val):
                    if init:
                        strn = strn + "(\'" + str(k[0])+"\')"
                        init = False
                    else:
                        strn = strn + "," + "(\'" + str(k[0])+"\')"
            else:
                for k in sorted(val):
                    if init:
                        strn = strn + str(tuple(k))
                        init = False
                    else:
                        strn = strn + "," + str(tuple(k))
            strn = strn + "\n\n"
            f.write(strn)


        f.write("\nFrequent Itemsets:\n")
        f_ilist = defaultdict(list)
        for x in freq_item_set:
            f_ilist[len(x)].append((sorted(tuple(x))))

        for k, val in sorted(f_ilist.items()):
            #print(k, sorted(val))
            init = True
            strn = ""
            if k==1:
                for k in sorted(val):
                    if init:
                        strn = strn + "(\'" + str(k[0])+"\')"
                        init = False
                    else:
                        strn = strn + "," + "(\'" + str(k[0])+"\')"
            else:
                for k in sorted(val):
                    if init:
                        strn = strn + str(tuple(k))
                        init = False
                    else:
                        strn = strn + "," + str(tuple(k))
            strn = strn + "\n\n"
            f.write(strn)

write_output(candidates, freq_item_set)